In [ ]:
from model import *
import numpy as np
from tensorflow.keras.optimizers import Adam, RMSprop
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model


def generate_real_samples(dataset, ground_trud_ds, n_samples, patch_size):
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    gt = ground_trud_ds[ix]
    y = np.ones((n_samples, patch_size, patch_size, 1))
    return X, y, gt


def generate_fake_samples(g_model, dataset, patch_size):
    w_noise = np.random.normal(0, 1, (dataset.shape[0], 14, 14, 1024))
    X = g_model.predict([dataset, w_noise])
    y = np.zeros((len(X), patch_size, patch_size, 1))
    return X, y


def sample_images(generator, source, target, idx):
    print(target.shape)
    target = np.uint8(target * 127.5 + 127.5)
    w_noise = np.random.normal(0, 1, (1, 14, 14, 1024))
    predicted = generator.predict([source, w_noise])
    im = np.uint8(predicted[0, ...] * 127.5 + 127.5)
    im_source = np.uint8(source[0, ...] * 255)
    print(im_source.shape)
    im_c = np.concatenate((np.squeeze(im, axis=-1), np.squeeze(target, axis=-1),
                           im_source[..., 0], im_source[..., 1]), axis=1)
    plt.imsave('./outputs/sketch_conversion' + str(idx) + '.png', im_c, cmap='terrain')



def train_gan():
    data = np.load('training_data4.npz')
    XTrain = data['x']
    YTrain = data['y']
    input_shape_gen = (XTrain.shape[1], XTrain.shape[2], XTrain.shape[3])
    input_shape_disc = (YTrain.shape[1], YTrain.shape[2], YTrain.shape[3])
    print(input_shape_gen,input_shape_disc)

    terrain_generator = UNet(input_shape_gen)
    terrain_discriminator = patch_discriminator(input_shape_disc)
    optd = Adam(0.0001, 0.5)
    terrain_discriminator.compile(loss='binary_crossentropy', optimizer=optd)
    composite_model = mount_discriminator_generator(
        terrain_generator, terrain_discriminator, input_shape_gen)
    composite_model.compile(
        loss=[
            'binary_crossentropy', 'mae'], loss_weights=[
            1, 3], optimizer=optd)

    n_epochs, n_batch, = 150, 20
    bat_per_epo = int(len(XTrain) / n_batch)
    patch_size = 15
    n_steps = bat_per_epo * n_epochs
    min_loss = 999
    avg_loss = 0
    avg_dloss = 0
    for i in range(n_steps):
        X_real, labels_real, Y_target = generate_real_samples(XTrain, YTrain, n_batch, patch_size)
        Y_target[np.isnan(Y_target)] = 0
        X_real[np.isnan(X_real)] = 0
        
        # mask 
        mask = np.random.uniform(low=0.0, high=1.0, size=X_real.shape)>0.75 # mask 75 %
        X_real = X_real* mask

        Y_fake, labels_fake = generate_fake_samples(terrain_generator, X_real, patch_size)
        w_noise = np.random.normal(0, 1, (n_batch, 14, 14, 1024))
        losses_composite = composite_model.train_on_batch(
            [X_real, w_noise], [labels_real, Y_target])

        loss_discriminator_fake = terrain_discriminator.train_on_batch(
            [Y_fake, X_real], labels_fake)
        loss_discriminator_real = terrain_discriminator.train_on_batch(
            [Y_target, X_real], labels_real)
        d_loss = (loss_discriminator_fake + loss_discriminator_real) / 2
        avg_dloss = avg_dloss + (d_loss - avg_dloss) / (i + 1)
        avg_loss = avg_loss + (losses_composite[0] - avg_loss) / (i + 1)

        if i % 100 == 0:
            print(i,'total loss:' + str(avg_loss) + ' d_loss:' + str(avg_dloss))
            sample_images(terrain_generator, X_real[0:1, ...], Y_target[0, ...], i)
        if i % 500 == 0:
            terrain_discriminator.save('terrain_discriminator' + str(i) + '.h5', True)
            terrain_generator.save('terrain_generator' + str(i) + '.h5', True)

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

train_gan()